In [39]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

In [40]:
# Load the dataset using Pandas
data = pd.read_csv('../data/diabetes.csv')

In [41]:
# For x: Extract out the dataset from all the rows (all samples) and all columns except last column (all features). 
# For y: Extract out the last column (which is the label)
# Convert both to numpy using the .values method
x = data.iloc[:,0:-1].values
y_string= list(data.iloc[:,-1])

In [42]:
# Lets have a look some samples from our data
print(x[:3])
print(y_string[:3])

[[  6.  148.   72.   35.    0.   33.6  50. ]
 [  1.   85.   66.   29.    0.   26.6  31. ]
 [  8.  183.   64.    0.    0.   23.3  32. ]]
['positive', 'negative', 'positive']


In [43]:
# Our neural network only understand numbers! So convert the string to labels
y_int = []
for string in y_string:
    if string == 'positive':
        y_int.append(1)
    else:
        y_int.append(0)

In [44]:
# Now convert to an array
y = np.array(y_int, dtype = 'float64')

### $x^{\prime}=\frac{x-\mu}{\sigma}$

In [45]:
# Feature Normalization. All features should have the same range of values (-1,1)
sc = StandardScaler()
x = sc.fit_transform(x)
print(x.shape)
print(x[0:3])

(768, 7)
[[ 0.63994726  0.84832379  0.14964075  0.90726993 -0.69289057  0.20401277
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575  0.53090156 -0.69289057 -0.68442195
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 -1.28821221 -0.69289057 -1.10325546
  -0.10558415]]


In [46]:
# Now we convert the arrays to PyTorch tensors
x = torch.tensor(x)
# We add an extra dimension to convert this array to 2D
y = torch.tensor(y).unsqueeze(1)

In [47]:
print(x.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


In [48]:
class Dataset(Dataset):

    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self,index):
        # Get one item from the dataset
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [49]:
dataset = Dataset(x,y)

In [50]:
len(dataset)

768

In [51]:
# Load the data to your dataloader for batch processing and shuffling
train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=32,
                                           shuffle=True)

In [52]:
# Let's have a look at the data loader
print("There is {} batches in the dataset".format(len(train_loader)))
for (x,y) in train_loader:
    print("For one iteration (batch), there is:")
    print("Data:    {}".format(x.shape))
    print("Labels:  {}".format(y.shape))
    break

There is 24 batches in the dataset
For one iteration (batch), there is:
Data:    torch.Size([32, 7])
Labels:  torch.Size([32, 1])


![demo](https://user-images.githubusercontent.com/30661597/60379583-246e5e80-9a68-11e9-8b7f-a4294234c201.png)

In [53]:
# Now let's build the above network
class Model(nn.Module):
    def __init__(self, input_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
    # Forward pass
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

$H_{p}(q)=-\frac{1}{N} \sum_{i=1}^{N} y_{i} \cdot \log \left(p\left(y_{i}\right)\right)+\left(1-y_{i}\right) \cdot \log \left(1-p\left(y_{i}\right)\right)$


cost = -(Y * torch.log(hypothesis) + (1 - Y) * torch.log(1 - hypothesis)).mean()

In [56]:
# Create the network (an object of the Net class)
model = Model(x.shape[1]) # 7
print(model.register_parameter)
#In Binary Cross Entropy: the input and output should have the same shape 
#size_average = True --> the losses are averaged over observations for each minibatch
criterion = torch.nn.BCELoss(size_average=True)   
# We will use SGD with momentum with a learning rate of 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

<generator object Module.parameters at 0x7ff516c78bd0>


In [55]:
# Train the network 
num_epochs = 200
for epoch in range(num_epochs):
    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        # Feed Forward
        output = model(inputs)
        # Loss Calculation
        loss = criterion(output, labels)
        # Clear the gradient buffer (we don't want to accumulate gradients)
        optimizer.zero_grad()
        # Backpropagation 
        loss.backward()
        # Weight Update: w <-- w - lr * gradient
        optimizer.step()
        
    #Accuracy
    # Since we are using a sigmoid, we will need to perform some thresholding
    output = (output>0.5).float()
    # Accuracy: (output == labels).float().sum() / output.shape[0]
    accuracy = (output == labels).float().mean()
    # Print statistics 
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss, accuracy))

Epoch 1/200, Loss: 0.639, Accuracy: 0.625
Epoch 2/200, Loss: 0.547, Accuracy: 0.688
Epoch 3/200, Loss: 0.493, Accuracy: 0.750
Epoch 4/200, Loss: 0.472, Accuracy: 0.719
Epoch 5/200, Loss: 0.495, Accuracy: 0.688
Epoch 6/200, Loss: 0.477, Accuracy: 0.781
Epoch 7/200, Loss: 0.395, Accuracy: 0.844
Epoch 8/200, Loss: 0.339, Accuracy: 0.906
Epoch 9/200, Loss: 0.535, Accuracy: 0.719
Epoch 10/200, Loss: 0.541, Accuracy: 0.719
Epoch 11/200, Loss: 0.444, Accuracy: 0.812
Epoch 12/200, Loss: 0.568, Accuracy: 0.719
Epoch 13/200, Loss: 0.679, Accuracy: 0.594
Epoch 14/200, Loss: 0.465, Accuracy: 0.750
Epoch 15/200, Loss: 0.510, Accuracy: 0.781
Epoch 16/200, Loss: 0.556, Accuracy: 0.688
Epoch 17/200, Loss: 0.309, Accuracy: 0.875
Epoch 18/200, Loss: 0.417, Accuracy: 0.812
Epoch 19/200, Loss: 0.349, Accuracy: 0.875
Epoch 20/200, Loss: 0.419, Accuracy: 0.750
Epoch 21/200, Loss: 0.469, Accuracy: 0.781
Epoch 22/200, Loss: 0.390, Accuracy: 0.844
Epoch 23/200, Loss: 0.606, Accuracy: 0.656
Epoch 24/200, Loss: 